<a href="https://colab.research.google.com/github/Karimkhab/introduction-to-machine-learning/blob/main/Convs_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from abc import ABC, abstractmethod

In [ ]:

def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

In [ ]:
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass

In [ ]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [ ]:
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    print(conv2d_out)
    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)

In [ ]:
import numpy as np

In [ ]:
# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        # input: (B, Cin, Hin, Win)
        B, Cin, Hin, Win = input_tensor.shape
        Cout = self.kernel.shape[0]
        K = self.kernel.shape[2]
        S = self.stride
        P = 0  # в твоём задании padding=0

        _, _, Hout, Wout = calc_out_shape(input_tensor.shape, Cout, K, S, P)
        output = torch.zeros((B, Cout, Hout, Wout), dtype=input_tensor.dtype)

        # PyTorch Conv2d = cross-correlation:
        # out[b, oc, oy, ox] = sum_{ic, ky, kx} input[b, ic, oy*S+ky, ox*S+kx] * kernel[oc, ic, ky, kx]
        for b in range(B):
            for oc in range(Cout):
                for oy in range(Hout):
                    for ox in range(Wout):
                        acc = 0.0
                        iy0 = oy * S
                        ix0 = ox * S
                        for ic in range(Cin):
                            for ky in range(K):
                                for kx in range(K):
                                    acc += (
                                        input_tensor[b, ic, iy0 + ky, ix0 + kx]
                                        * self.kernel[oc, ic, ky, kx]
                                    )
                        output[b, oc, oy, ox] = acc

        return output

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):

In [ ]:
print(test_conv2d_layer(Conv2dLoop))

In [2]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


In [3]:
class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        # print(self.kernel, self.kernel.shape)
        # print(torch_input, torch_input.shape)
        In_shapes = torch_input.shape
        K = self.kernel[0]
        K_shapes = self.kernel.shape
        kernel_unsqueezed = torch.tensor([[[0]*(In_shapes[-1]*In_shapes[-2])]*In_shapes[-2]]*K_shapes[1]).float() # Реализуйте функцию, возвращающую преобразованный кернел.
        # print(kernel_unsqueezed)
        # print(K_shapes)

        for k in range(K_shapes[1]):
            bias = 0
            kernel = K[k]
            # print(kernel)
            string = []
            for row in range(K_shapes[-1]):
                string += list(kernel[row]) + [torch.tensor(0)]
            string = torch.tensor(string)[:-1]
            # print(string, len(string))

            Un_shapes = kernel_unsqueezed.shape
            # print(Un_shapes)
            new = kernel_unsqueezed[k]
            # print(new)
            for row in range(Un_shapes[-2]):
                new[row][bias:bias+len(string)] = string
                if row%2==0: bias+=1
                else: bias+=3
            kernel_unsqueezed[k] = new
            # print(kernel_unsqueezed)
            # print(new, new.shape)



            # print('\n\n\n')
        print(kernel_unsqueezed)
        return kernel_unsqueezed

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dMatrix))

In [11]:
class Conv2dMatrix(ABCConv2d):
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        B, Cin, H, W = torch_input.shape
        Cout, _, K, _ = self.kernel.shape
        S = self.stride
        print(B, Cin, H, W, '\n', Cout, K,'\n', S)

        # W_big: (Cout*Hout*Wout, Cin*H*W)
        W_big = torch.zeros((Cout * output_height * output_width, Cin * H * W),
                            dtype=torch_input.dtype)
        print(W_big, W_big.shape)

        row = 0
        for oc in range(Cout):
            for oy in range(output_height):
                for ox in range(output_width):
                    print(ox, oy)
                    iy0 = oy * S
                    ix0 = ox * S
                    for ic in range(Cin):
                        for ky in range(K):
                            for kx in range(K):
                                iy = iy0 + ky
                                ix = ix0 + kx
                                col = ic * (H * W) + iy * W + ix
                                W_big[row, col] = self.kernel[oc, ic, ky, kx]
                    row += 1
        print(W_big, W_big.shape)
        return W_big

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

In [12]:
print(test_conv2d_layer(Conv2dMatrix))

2 3 4 4 
 1 3 
 2
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]) torch.Size([1, 48])
0 0
tensor([[ 0.,  1.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  1.,  2.,  1.,  0.,  0.,  3.,  3.,  0.,  0.,  1., 10.,  0.,
          0.,  0.,  0.,  0., 10., 11., 12.,  0., 13., 14., 15.,  0., 16., 17.,
         18.,  0.,  0.,  0.,  0.,  0.]]) torch.Size([1, 48])
True


In [36]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


In [85]:
class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        B, N, H, W = self.kernel.shape

        converted_kernel = torch.zeros((B, N*H*W))  # Реализуйте преобразование кернела.
        S_reshape = H*W
        for b in range(B):
            for k in range(N):
                converted_kernel[b][k*S_reshape:k*S_reshape+S_reshape] = self.kernel[b][k].reshape((1,S_reshape))

        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        B, N, H, W = torch_input.shape
        _, _, Kh, Kw = self.kernel.shape

        calc_out_size = lambda s: (s-self.kernel_size)//self.stride + 1

        H_out, W_out = calc_out_size(H), calc_out_size(W)

        converted_input = torch.zeros((N*Kh*Kw, B*H_out*W_out)) # Реализуйте преобразование входа.

        col = 0
        for b in range(B):
            for oy in range(output_height):
                for ox in range(output_width):
                    ix = ox * self.stride
                    iy = oy * self.stride
                    row = 0
                    for c in range(N):
                        patch = torch_input[b, c, iy:iy+Kh, ix:ix+Kw].reshape(-1)   # (Kh*Kw,)
                        converted_input[row:row + Kh*Kw, col] = patch
                        row += Kh * Kw

                    col += 1

        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(1,0).view(torch_input.shape[0],
                                                     self.out_channels,
                                                     output_height,
                                                     output_width)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dMatrixV2))

In [86]:
print(test_conv2d_layer(Conv2dMatrixV2))

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
          [68., 69., 70., 71.],
          [72., 73., 74., 75.],
          [76., 77., 78., 79.]],

         [[80., 81., 82., 83.],
          [84., 85., 86., 87.],
          [88., 89., 90., 91.],
          [92., 93., 94., 95.]]]]) torch.Size([2, 3, 4, 4])
tensor([ 0.,  1.,  2.,  4.,  5.,  6.,  8.,  9., 10.])
tensor([16., 17., 18., 20., 21., 22., 24., 25., 26.])
tensor([32., 33., 34., 36., 37., 38., 40., 41., 42.])
tensor([48., 49., 50., 52., 53